In [91]:
import os
%pwd

'C:\\Users\\SGU\\Projects\\mlops-with-mlflow'

In [92]:
os.chdir(r"C:\Users\SGU\Projects\mlops-with-mlflow")
%pwd

'C:\\Users\\SGU\\Projects\\mlops-with-mlflow'

In [93]:
import pandas as pd

In [94]:
data = pd.read_csv(r"artifacts\data_ingestion\winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [95]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [96]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [97]:
data.shape

(1599, 12)

In [98]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [99]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_path: Path
    status_file: Path
    all_schema: dict
    
    



In [100]:
from mlProject.utils.constants import CONFIG_FILE_PATH, SCHEMA_FILE_PATH, PARAMS_FILE_PATH
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self) -> None:
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.schema = read_yaml(SCHEMA_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        
        print(self.config)
        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        create_directories([config.root_dir])
        return DataValidationConfig(
            config.root_dir,
            config.unzip_data_path,
            config.status_file,
            self.schema.columns
        )
        
        

In [101]:
from mlProject import logger

class DataValidation:
    def __init__(self, config: DataValidationConfig) -> None:
        self.config = config
        
    def validate_all_columns(self)-> bool:
        try:
            status = True
            data =  pd.read_csv(self.config.unzip_data_path)
            all_cols = list(data.columns)
            
            for col in all_cols:
                if col not in self.config.all_schema:
                    status = False
                    with open(self.config.status_file, "w") as f:
                        f.write(f"failed validation with column: {col} \n")
                    break
                else:
                    with open(self.config.status_file, "w") as f:
                        f.write(f"validation passed with column: {col}")
                # if self.config.all_schema[col] != type(col):
                #     status = False
                #     with open(self.config.status_file, "w") as f:
                #         f.write(f"failed validation with column: {col}")
                #     break
            return status
        except Exception as e:
            raise e
                

In [102]:
try:
    config = ConfigurationManager()
    validation_cfg = config.get_data_validation_config()
    obj = DataValidation(validation_cfg)
    obj.validate_all_columns()
except Exception as e:
    raise e

[2023-08-11 23:24:02,953: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-11 23:24:02,960: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-11 23:24:02,960: INFO: common: yaml file: params.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/entbappy/Branching-tutorial/raw/master/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_path': 'artifacts/data_ingestion/winequality-red.csv', 'status_file': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation'}}
[2023-08-11 23:24:02,964: INFO: common: created directory: artifacts]
[2023-08-11 23:24:02,965: INFO: common: created directory: artifacts\data_validation]
